In [ ]:
import glob
import torch
from sklearn.manifold import TSNE
import glob
import json
import pandas as pd
import seaborn as sns
import argparse
import matplotlib.pyplot as plt

In [ ]:
embed_res = glob.glob("./results/*/*.pt")
embed_meta_res = glob.glob("./results/*/*/*/metadata.tsv")

In [ ]:
for embed, meta in zip(embed_res, embed_meta_res):
    print("plotting ", embed)
    embed_ckpt = torch.load(embed).numpy()
    cell_labels = []
    with open(meta) as handle:
        for line in handle.readlines():
            cell_labels.append(int(line.strip().split("-")[0]))
    
    with open(embed.replace("embed.pt", 'id2cell.json'),'r') as handle:
        id2cell = json.load(handle)
    cell_labels = [id2cell[str(lb)] for lb in cell_labels]
    tsne = TSNE(verbose=1)
    embed_transform = tsne.fit_transform(embed_ckpt)
    
    embed_df = pd.DataFrame(embed_transform)
    embed_df.columns = ['tsne_one', 'tsne_two']
    embed_df['cell_type'] = cell_labels
    plt.figure(figsize=(16,10))
    sns.scatterplot(x='tsne_one', y='tsne_two',\
                   data=embed_df, hue='cell_type',\
                   palette=sns.color_palette("hls", 9))
    plt.savefig(embed.replace('.pt', 'tsne_embed'))

In [1]:
import anndata
import os
import scanpy as sc

In [2]:
data_ref = './data/baron_2016h.h5ad'
data_trg = "./data/xin_2016.h5ad"
adata_ref = anndata.read_h5ad(data_ref)
adata_trg = anndata.read_h5ad(data_trg)

In [3]:
sc.pp.filter_genes(adata_ref, min_counts=1)

In [21]:
import torch
torch.tensor(adata_ref.X)

TypeError: sparse matrix length is ambiguous; use getnnz() or shape[0]

In [4]:
shared_genes = adata_ref.var_names[adata_ref.var_names.isin(adata_trg.var_names)]

In [5]:
adata_ref = adata_ref[:, adata_ref.var_names.isin(shared_genes)]
adata_trg = adata_trg[:, adata_trg.var_names.isin(shared_genes)]

In [6]:
adata_ref = adata_ref[:, adata_ref.var_names.argsort()].copy()
adata_trg = adata_trg[:, adata_trg.var_names.argsort()].copy()
assert all(adata_ref.var_names == adata_trg.var_names)

In [7]:
from utils import select_gene
gene_mask = select_gene(adata_ref.X, num_gene=1000, threshold=0)

Chosen offset: 0.45


array([False, False,  True, ..., False, False, False])